# **LIBRARY IMPORT**

In [ ]:
### Imports

import numpy as np #used for working with arrays
import cv2 as cv
import random #generate random numbers
import pickle
import glob # used to retrieve files/pathnames matching a specified pattern
import os #operating system
import shutil #high-level operation on a file like a copy, create, and remote operation on the file.
import tensorflow as tf
import tensorflow.keras as keras #for deep learning
import matplotlib.pyplot as plt #for plotting functions
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout,AveragePooling2D, Flatten #for classification layer
from sklearn.model_selection import train_test_split  #for splitting data
!pip install livelossplot #live plotting of loss and accuracy graph
from livelossplot.keras import PlotLossesCallback
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report


# **TRAINING PARAMETERS**

In [ ]:
selectedModel = "resnet50";
neurons = [4096,0.8,1024, 256]
dataPath = '../input/traindata/' ## Folder for dataset

ext = [".jpg",".JPG",".jpeg",".JPEG",".png",".PNG"]
learningRate = 0.00001; #hyperparameter that controls how much to change the model in response to the estimated error each time the model weights are updated
test_split_size=0.20
validation_split_size=0;
maxEpochs = 500; 
useAllSamples = True ## True to Consume All Traindata per Epoch (for feature crucial classification like defects)
batchSize = 16 # number of training samples to work through before the model's internal parameters are updated.
fixedStepSize = 2 ## If useAllSamples is False, get no. steps per epoch7
enableEarlyStop = True; #stop training once the model performance stops improving on a hold out validation dataset
estype = "accuracy"
patience = 5
layersToUnfreeze = 0

## Data Augment Parameters
rotAngle = 0
flipHorizontal = False
flipVertical = False


import os
for root, dirs, files in os.walk(dataPath):
    break;

numClasses = len(dirs);




# **MODEL TRAINER**

In [ ]:
#from tensorflow.keras.preprocessing import image
if selectedModel == "vgg16":
    imgsz = 224; ## 224 resnet and vgg, 299 for inception
    from tensorflow.keras.applications.vgg16 import preprocess_input
    sourceModel = keras.applications.VGG16
elif selectedModel == "inceptionv3":
    imgsz = 299; ## 224 resnet and vgg, 299 for inception
    from tensorflow.keras.applications.inception_v3 import preprocess_input
    sourceModel = keras.applications.InceptionV3
elif selectedModel == "resnet50":
    imgsz = 224; ## 224 resnet and vgg, 299 for inception
    from tensorflow.keras.applications.resnet50 import preprocess_input
    sourceModel = keras.applications.ResNet50


    
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping


# Adam is an optimization algorithm, replacing the very old Gradient Descent.


modelName = 'model_' + selectedModel + '.model';


#imports the model and discards the classification layer.
base_model=sourceModel(weights='imagenet', \
                          include_top=False, \
                          input_shape = (imgsz,imgsz,3)); 


#Show Architecture
for i,layer in enumerate(base_model.layers):
    print(i,layer.name,layer.trainable)
    

## To Check Last Layer
#base_model.layers[-1]


x=base_model.output

addedLayers = 0;
# if selectedModel == "vgg16":

x = GlobalAveragePooling2D()(x); addedLayers = addedLayers + 1;

for neuron in neurons:
  if neuron<1:
    x=Dropout(neuron)(x)
  else:
    x=Dense(neuron,activation='relu')(x) 
  addedLayers = addedLayers + 1

preds=Dense(numClasses,activation='softmax')(x); addedLayers = addedLayers + 1;

# elif selectedModel == "inceptionv3":
#     x = GlobalAveragePooling2D()(x); addedLayers = addedLayers + 1;
#     preds=Dense(numClasses,activation='softmax')(x); addedLayers = addedLayers + 1;
# elif selectedModel == "resnet50":  
#x = GlobalAveragePooling2D()(x); addedLayers = addedLayers + 1;
#preds=Dense(numClasses,activation='softmax')(x); addedLayers = addedLayers + 1;

    


model=Model(inputs=base_model.input,outputs=preds)

                  
              
for layer in model.layers[0:-(addedLayers+layersToUnfreeze)]:
    layer.trainable = False

#Show Architecture
for i,layer in enumerate(model.layers):
    print(i,layer.name,layer.trainable)

    
## This datagen is used only for viewing, prior to preprocessing
viewing_datagent=ImageDataGenerator(rotation_range=rotAngle,
                                horizontal_flip=flipHorizontal,
                                vertical_flip=flipVertical,
                                validation_split=validation_split_size)

## Official Training Datagen, PreProcessed
if rotAngle == 0:
    train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
                                    #horizontal_flip=flipHorizontal,
                                    #vertical_flip=flipVertical,
                                    #validation_split=validation_split_size) 
else:
    train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)
                                    #rotation_range=rotAngle,
                                    #horizontal_flip=flipHorizontal,
                                    #vertical_flip=flipVertical,
                                    #validation_split=validation_split_size) 

print("Training Dataset:");
train_generator=train_datagen.flow_from_directory(dataPath,
                                                target_size=(imgsz,imgsz),
                                                color_mode='rgb',
                                                batch_size=batchSize,
                                                class_mode='categorical',
                                                shuffle=False,
                                                subset='training');
""""
print("Test Validation Dataset:");
validation_generator=train_datagen.flow_from_directory(dataPath,
                                                target_size=(imgsz,imgsz),
                                                color_mode='rgb',
                                                batch_size=batchSize,
                                                class_mode='categorical',
                                                subset='validation',
                                                shuffle=False);"""

print("Sample Viewing Dataset:");
## Only for Viewing Unprocessed Images
view_generator=viewing_datagent.flow_from_directory(dataPath,
                                                target_size=(imgsz,imgsz),
                                                color_mode='rgb',
                                                batch_size=batchSize,
                                                class_mode='categorical',
                                                shuffle=False);

## Show 5 Sample Images
x,y = view_generator.next() #returns the next item
for i in range(0,min([batchSize,5])):
    img = x[i]
    plt.imshow(img.astype(np.uint8))
    plt.show()
    
                                                  

adamopt = optimizers.Adam(lr=learningRate);
model.compile(optimizer=adamopt, \
              loss='categorical_crossentropy', \
              metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be validation accuracy

if useAllSamples:
   step_size_train=train_generator.n//train_generator.batch_size
else:
    step_size_train=fixedStepSize

if enableEarlyStop:
    es = EarlyStopping(monitor=estype,mode='auto', patience=patience)
    
    history = model.fit_generator(generator=train_generator,
                      steps_per_epoch=step_size_train,
                      epochs=maxEpochs,
                      callbacks=[PlotLossesCallback(),es]);
else:
    
    history = model.fit_generator(generator=train_generator,
                      steps_per_epoch=step_size_train,
                      epochs=maxEpochs);




# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


### Save Model
tf.keras.models.save_model(
    model,
    modelName,
    overwrite=True,
    include_optimizer=True
)
model.save("resnet50_novali.h5")
#### Save Label Names
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels);

fid = open('label_' + selectedModel + '.sav','wb');
pickle.dump(labels, fid);
fid.close();


print("Model successfully trained and saved.");


In [ ]:
print("Training Accuracy of the model is - " , model.evaluate(train_generator)[1]*100 , "%")
#print("Validation Accuracy of the model is - " , model.evaluate(validation_generator)[1]*100 , "%")

# **MODEL TESTING**

In [ ]:
#from tensorflow.keras.preprocessing import image
if selectedModel == "vgg16":
    imgsz = 224; ## 224 resnet and vgg, 299 for inception
    from tensorflow.keras.applications.vgg16 import preprocess_input
    sourceModel = keras.applications.VGG16
elif selectedModel == "inceptionv3":
    imgsz = 299; ## 224 resnet and vgg, 299 for inception
    from tensorflow.keras.applications.inception_v3 import preprocess_input
    sourceModel = keras.applications.InceptionV3
elif selectedModel == "resnet50":
    imgsz = 224; ## 224 resnet and vgg, 299 for inception
    from tensorflow.keras.applications.resnet50 import preprocess_input
    sourceModel = keras.applications.ResNet50


modelName = "model_" + selectedModel + ".model";


testmodel = tf.keras.models.load_model("resnet50_novali.h5")
#    custom_objects=None,
#    compile=True
#)

f = open("label_" + selectedModel + ".sav",'rb');labels = pickle.load(f);f.close();
labels = list(labels.values());

X = [];
y = [];


for label in labels:

    fileList = []
    for ctrext in range(0,len(ext)):
        
        fileList.extend(glob.glob(dataPath[:-1] + "test" + "/" + label + "/*" + ext[ctrext]))
    
    
    
    for thisFile in fileList:
        

        ## CV IMAGE LOAD
        img = cv.imread(thisFile);
        img = cv.cvtColor(img,cv.COLOR_BGR2RGB);
        
      
        img = cv.resize(img,(imgsz,imgsz));
        
        
        img = preprocess_input(img);
        
        X.append(img);
        y.append(labels.index(label));
        
    

y_preds = testmodel.predict(np.array(X));
y_pred = [];
for y_preds_i in y_preds:
    y_pred.append(np.argmax(y_preds_i));
#y_pred = testmodel.predict(X);

accuracy = sum(np.equal(y_pred,y)) / len(y) * 100

confmat = confusion_matrix(y, y_pred)
disp=ConfusionMatrixDisplay(confusion_matrix=confmat, display_labels=labels)

disp.plot(xticks_rotation='vertical',values_format='')
plt.show()



print()
print("RANK 1 Accuracy Against Test Data: " + "{:.2f}".format(accuracy) + " %");




print()
print('CONFUSION MATRIX:')
print(confmat)

print(labels);

print();
print("CLASSIFICATION REPORT:")
print(classification_report(y, y_pred))





## SHOW OUTPUT BREAKDOWN
print()
print("OUTPUT DISTRIBUTION:")
outctr = 0;
total = sum(sum(confmat));
for col in confmat.T:
    
    totalcol = sum(col)
    
    print(labels[outctr] + ": " + ("%1.2f") % ((totalcol / total) * 100) + "%")
    
    outctr = outctr + 1;


# **TEST ONE**

In [ ]:

#from tensorflow.keras.preprocessing import image
if selectedModel == "vgg16":
    imgsz = 224; ## 224 resnet and vgg, 299 for inception
    from tensorflow.keras.applications.vgg16 import preprocess_input
    sourceModel = keras.applications.VGG16
elif selectedModel == "inceptionv3":
    imgsz = 299; ## 224 resnet and vgg, 299 for inception
    from tensorflow.keras.applications.inception_v3 import preprocess_input
    sourceModel = keras.applications.InceptionV3
elif selectedModel == "resnet50":
    imgsz = 224; ## 224 resnet and vgg, 299 for inception
    from tensorflow.keras.applications.resnet50 import preprocess_input
    sourceModel = keras.applications.ResNet50

    
    

modelName = "model_" + selectedModel + ".model";


testmodel = tf.keras.models.load_model(
    modelName)
#    custom_objects=None,
#    compile=True
#)

f = open("label_" + selectedModel + ".sav",'rb');labels = pickle.load(f);f.close();
labels = list(labels.values());

X = [];
y = [];


idx = random.randint(0,len(labels)-1)
label = labels[idx]

imgFiles = []
for ctrext in range(0,len(ext)):
    imgFiles.extend(glob.glob(dataPath[:-1] + "test" + "/" + label + "/*" + ext[ctrext]))

idxFile = random.randint(0,len(imgFiles)-1)

filepath = imgFiles[idxFile]
pos1 = filepath.find("/")
pos2 = filepath.find("/",pos1+1)
actualClass = filepath[pos1+1:pos2]

## CV IMAGE LOAD
img = cv.imread(filepath);
img = cv.cvtColor(img,cv.COLOR_BGR2RGB);


plt.title("Input Image")
plt.imshow(img)
plt.show()

img = cv.resize(img,(imgsz,imgsz));
img = preprocess_input(img);

y_pred = testmodel.predict(np.array([img]));
print("Image Classified as: ",labels[np.argmax(y_pred[0])])
print("Actual Class:",actualClass)
print("Classification Score: ",max(y_pred[0]))